# Task 4 #
================

На предыдущих этапах ты занимался просто подготовкой данных. Ты не получил никакой аналитической информации на основе имеющихся данных. Пришло время приступить к анализу. Помни, что, согласно нашему предположению, пользователи приступили бы к работе над лабораторными заданиями раньше, если бы они просмотрели ленту новостей? Это означает, что ключевой метрикой для нас является период времени (delta) между датой начала работы пользователя над лабораторным заданием (первого коммита) и сроком сдачи лабораторного задания. Мы будем смотреть на то, меняется ли она в зависимости от просмотра страницы.

Что тебе нужно сделать в этом упражнении:

In [1]:
import sqlite3
import pandas as pd

## Создай соединение с базой данных с помощью библиотеки sqlite3. ##
----------

In [2]:
conn = sqlite3.connect("../data/checking-logs.sqlite")
cursor = conn.cursor()

## Получи схему таблицы test. ##
--------

In [3]:
# Получаем схему таблицы test
schema = pd.read_sql_query("PRAGMA table_info(test)", conn)
print(schema)

   cid             name       type  notnull dflt_value  pk
0    0              uid       TEXT        0       None   0
1    1          labname       TEXT        0       None   0
2    2  first_commit_ts  TIMESTAMP        0       None   0
3    3    first_view_ts  TIMESTAMP        0       None   0


## Получи только первые 10 строк таблицы test, чтобы проверить, как она выглядит ##
--------

In [4]:
# Получаем первые 10 строк таблицы test
first_10 = pd.read_sql_query("SELECT * FROM test LIMIT 10", conn)
print(first_10)

       uid   labname             first_commit_ts               first_view_ts
0   user_1    laba04  2020-04-26 17:06:18.462708  2020-04-26 21:53:59.624136
1   user_1   laba04s  2020-04-26 17:12:11.843671  2020-04-26 21:53:59.624136
2   user_1    laba05  2020-05-02 19:15:18.540185  2020-04-26 21:53:59.624136
3   user_1    laba06  2020-05-17 16:26:35.268534  2020-04-26 21:53:59.624136
4   user_1   laba06s  2020-05-20 12:23:37.289724  2020-04-26 21:53:59.624136
5   user_1  project1  2020-05-14 20:56:08.898880  2020-04-26 21:53:59.624136
6  user_10    laba04  2020-04-25 08:24:52.696624  2020-04-18 12:19:50.182714
7  user_10   laba04s  2020-04-25 08:37:54.604222  2020-04-18 12:19:50.182714
8  user_10    laba05  2020-05-01 19:27:26.063245  2020-04-18 12:19:50.182714
9  user_10    laba06  2020-05-19 11:39:28.885637  2020-04-18 12:19:50.182714


In [5]:
# Получаем первые 10 строк таблицы deadlines
first_10 = pd.read_sql_query("SELECT * FROM deadlines LIMIT 10", conn)
print(first_10)

   index      labs   deadlines
0      0    laba04  1587945599
1      1   laba04s  1587945599
2      2    laba05  1588550399
3      4    laba06  1590364799
4      5   laba06s  1590364799
5      3  project1  1589673599


## Найди среди всех пользователей минимальное значение этой самой дельты (периода времени от даты первого коммита пользователя до срока сдачи соответствующего лабораторного задания), используя только один запрос. ##
-------------

* Для этого сджойни данные своей таблицы с данными таблицы deadlines.
* Период времени должен отображаться в часах.
* Не учитывай лабораторное задание project1 с более длительным сроком сдачи, поскольку оно будет выделяться на фоне других.
* Сохрани значение в датафрейме df_min с соответствующим uid.

In [6]:
query_min = f"""SELECT s1.uid, MIN(strftime('%s', s1.first_commit_ts) - s2.deadlines)/60/60 AS MIN_diff
            FROM (  SELECT * FROM test
                    WHERE labname != 'project1'
                    GROUP BY uid, labname) AS s1
            LEFT JOIN ( SELECT *
                        FROM deadlines )AS s2
            ON s1.labname = s2.labs;"""

# Используем метод read_sql_query библиотеки pandas для выполнения запроса и получения данных в виде DataFrame
df_min = pd.read_sql_query(query_min, conn)
df_min

,uid,MIN_diff
0,user_30,-202


## Выполни те же самые операции для максимального значения дельты, используя только один запрос. Название итогового датафрейма — df_max. ##
--------

In [7]:
query_max = f"""SELECT s1.uid, max(strftime('%s', s1.first_commit_ts) - s2.deadlines)/60/60 AS MAX_diff
            FROM (  SELECT * FROM test
                    WHERE labname != 'project1'
                    GROUP BY uid, labname) AS s1 
            LEFT JOIN ( SELECT *
                        FROM deadlines )AS s2 
            ON s1.labname = s2.labs;"""
             
df_max = pd.read_sql_query(query_max, conn)
df_max

,uid,MAX_diff
0,user_25,-2


## Выполни те же самые операции для среднего значения дельты, используя только один запрос. На этот раз ваш итоговый датафрейм не должен включать столбец uid; имя датафрейма — df_avg. ##
--------

In [8]:
query_avg = f"""SELECT  avg((strftime('%s', s1.first_commit_ts) - s2.deadlines)/60/60) AS avg_diff
            FROM (  SELECT * FROM test
                    WHERE labname != 'project1'
                    GROUP BY uid, labname) AS s1 
            LEFT JOIN ( SELECT *
                        FROM deadlines )AS s2 
            ON s1.labname = s2.labs;"""

df_avg = pd.read_sql_query(query_avg, conn)
df_avg

,avg_diff
0,-89.125


## Мы хотим проверить предположение о том, что у пользователей, посетивших ленту новостей всего несколько раз, период времени между датой первого коммита и сроком сдачи лабораторного задания меньше. Для этого тебе необходимо рассчитать коэффициент корреляции между количеством просмотров страниц и разницей между датами. ##
-------

* Используя только один запрос, создай таблицу со столбцами: uid, avg_diff, pageviews.
    - uid — это uid, существующие в таблице test.
    - avg_diff — среднее значение дельты (периода времени между датой первого коммита и сроком сдачи лабораторного задания) для каждого пользователя.
    - pageviews — количество посещений ленты новостей одним пользователем.
* Не учитывай лабораторное задание project1.
* Сохрани результаты в датафрейме views_diff.
* Используй метод corr() библиотеки Pandas для вычисления коэффициента корреляции между количеством просмотров и значением дельты.

In [9]:
query = f"""SELECT  s1.uid, s1.avg_diff , s2.pageviews
            FROM (  SELECT test.uid, avg((strftime('%s', test.first_commit_ts) - deadlines.deadlines)/60/60) AS avg_diff
                        FROM test 
                        LEFT JOIN deadlines 
                        on test.labname=deadlines.labs
                        WHERE test.labname != 'project1'
                        GROUP BY uid
                    ) AS s1 
            
            LEFT  JOIN ( SELECT uid, count(*) as pageviews
                            FROM pageviews 
                            where uid LIKE 'user_%'
                            GROUP BY uid
                        )AS s2 
            ON s1.uid = s2.uid;"""

views_diff = pd.read_sql_query(query, conn)

In [10]:
# Вычисляем коэффициент корреляции
correlation = views_diff[['pageviews','avg_diff']]
correlation.corr()

,pageviews,avg_diff
pageviews,1.000000,-0.279736
avg_diff,-0.279736,1.000000


## Закрой соединение с базой данных. ##
------

In [11]:
# Закрываем соединение
conn.close()